In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
from PIL import Image
import os
from tqdm import tqdm

def calculate_mean_std(data_path):
    channel_sum = np.zeros(3)
    channel_sum_squared = np.zeros(3)
    pixel_count = 0
    
    # 获取所有图片文件
    image_files = []
    for root, _, files in os.walk(data_path):
        for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png')):
                image_files.append(os.path.join(root, file))
    
    # 遍历所有图片并显示进度条
    for img_path in tqdm(image_files, desc="Processing images"):
        img = Image.open(img_path).convert('RGB')
        img = np.array(img) / 255.0  # 归一化到 [0,1]
        
        pixel_count += (img.shape[0] * img.shape[1])
        channel_sum += np.sum(img, axis=(0, 1))
        channel_sum_squared += np.sum(np.square(img), axis=(0, 1))
    
    # 计算均值
    mean = channel_sum / pixel_count
    
    # 计算标准差
    std = np.sqrt((channel_sum_squared / pixel_count) - np.square(mean))
    
    return mean, std

mean, std = calculate_mean_std(data_path)
print('均值: ', mean)
print('标准差: ', std)


Processing images: 100%|██████████| 27153/27153 [02:28<00:00, 182.43it/s]

均值:  [0.75885325 0.77881755 0.75984938]
标准差:  [0.24770346 0.23468548 0.26303263]
